In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import poisson

## 1. Classical liqudation problem with permanent and temporary price impact

**Model setup**

value function: $$H(t,x,S,q) = \sup_{\nu \in \mathscr{A}} \mathbb{E}_{t,x,S,q}[X_T^{\nu} + Q_T^{ \nu}(S_T^{\nu} - \alpha Q_T^{\nu}) - \phi \int_t^T(Q_u^{\nu})^2 du]$$

* $dX_t^{\nu} = (S_t^{\nu}-f(\nu_t)) \nu_t dt$
* $dS_t^{\nu} = -g(\nu_t) dt + \sigma dW_t, \quad S_0^{\nu}=S$
* $dQ_t^{\nu} = - \nu_t dt, \quad Q_0^{\nu}=q$

HJB equation: $$\partial_t H + \frac{1}{2}\sigma^2 \partial_{SS}H - \phi q^2 + \sup_{\nu} \{\nu (S-f(\nu)) \partial_x H - g(v) \partial_S H - \nu \partial_q H \} = 0$$
$$H(T,x,S,q) = x + q(S-\alpha q)$$

* Consider the linear case: $f(\nu) = k\nu, g(\nu)=b\nu$

From the setting, the analytical solution is: $$\nu_t^{*} = \gamma \frac{\xi e^{\gamma(T-t)}+e^{-\gamma(T-t)}}{\xi e^{\gamma(T-t)}-e^{-\gamma(T-t)}}Q_t^{\nu^{*}}$$ 

where $\gamma = \sqrt{\frac{\phi}{k}}, \xi = \frac{\alpha - \frac{1}{2}b + \sqrt{k\phi}}{\alpha - \frac{1}{2}b - \sqrt{k\phi}}$.

In [ ]:
# Simulate the data 

# Run 64 samples 
N_MC = 64

# Set parameters
# trade once per second in one day (approximately set trade hour in one day is 7)
T = 1
N = 100 
dt = float(T) / N
t = np.linspace(0,T,N+1)
# temporary price impact & permanent price impact
#k = 10 ** (-4)
#b = 2*10 ** (-4)
# terminal penalty
#alpha = 100 * k
# running penalty
#phi = 10 ** (-3)
# volatility of stock price 
#sigma = 1 
# initial price
S0 = 20
# initial inventory 
q = 1 

# parameters
b = 0.001
sigma = 0.1
k = 0.001
alpha = 0.01
phi = 0.01

dW = np.sqrt(dt)*np.random.randn(N_MC,N)
S = np.zeros([N_MC,len(t)])
v = np.zeros([N_MC,len(t)])
X = np.zeros([N_MC,len(t)])
Q = np.zeros([N_MC,len(t)])
Q[:,0] = q
S[:,0] = S0
X[:,0] = 0

In [ ]:
gamma = np.sqrt(phi/k)
epsilon = (alpha-0.5*b+np.sqrt(k*phi)) / (alpha-0.5*b-np.sqrt(k*phi))

# Euler Scheme 
for i in range(N+1):
    v[:,i] =  q * gamma * (epsilon*np.exp(gamma*(T-dt*i))+np.exp(-gamma*(T-dt*i))) / (epsilon*np.exp(gamma*T)-np.exp(-gamma*T))
for i in range(1,N+1):
    S[:,i] = S[:,i-1] + (-b*v[:,i-1])*dt + sigma*dW[:,i-1] 
    Q[:,i] = Q[:,i-1] - v[:,i-1]*dt
    X[:,i] = X[:,i-1] + (S[:,i-1]-k*v[:,i-1])*v[:,i-1]*dt
    
# Optimal strategy
optimal_v = v[0,:]

In [ ]:
# Plot
plt.figure(figsize=(15, 10))

plt.subplot(221)
for i in range(0,N_MC):
    plt.plot(t,S[i],linewidth=0.5)

plt.title("Stock price over time")
plt.xlabel("T")

plt.subplot(222)
for i in range(0,N_MC):
    plt.plot(t,X[i],linewidth=0.5)

plt.title("Revenue over time")
plt.xlabel("T")

plt.subplot(223)
for i in range(0,N_MC):
    plt.plot(t,v[i],linewidth=0.5)

plt.title("Optimal strategy over time")
plt.xlabel("T")
plt.ylabel("Trading Speed")


plt.subplot(224)
for i in range(0,N_MC):
    plt.plot(t,Q[i],linewidth=0.5)

plt.title("Inventory over time")
plt.xlabel("T")
plt.ylabel("Inventory")

plt.show()

In [ ]:
def change_parameter(phi,alpha):
    # Simulate the data 

    # Run 100 samples 
    N_MC = 100

    # Set parameters
    # trade once per second in one day (approximately set trade hour in one day is 7)
    T = 1
    N = 3600 * 7 
    dt = float(T) / N
    t = np.linspace(0,T,N+1)
    # temporary price impact & permanent price impact
    k = 10 ** (-4)
    b = 2*10 ** (-4)
    # volatility of stock price 
    sigma = 1 
    # initial price
    S0 = 20
    # initial inventory 
    q = 1 

    dW = np.sqrt(dt)*np.random.randn(N_MC,N)
    S = np.zeros([N_MC,len(t)])
    v = np.zeros([N_MC,len(t)])
    X = np.zeros([N_MC,len(t)])
    Q = np.zeros([N_MC,len(t)])
    Q[:,0] = q
    S[:,0] = S0
    X[:,0] = 0
    
    gamma = np.sqrt(phi/k)
    epsilon = (alpha-0.5*b+np.sqrt(k*phi)) / (alpha-0.5*b-np.sqrt(k*phi))

    # Euler Scheme 
    for i in range(N+1):
        v[:,i] =  q * gamma * (epsilon*np.exp(gamma*(T-dt*i))+np.exp(-gamma*(T-dt*i))) / (epsilon*np.exp(gamma*T)-np.exp(-gamma*T))
    for i in range(1,N+1):
        S[:,i] = S[:,i-1] + (-b*v[:,i-1])*dt + sigma*dW[:,i-1] 
        Q[:,i] = Q[:,i-1] - v[:,i-1]*dt
        X[:,i] = X[:,i-1] + (S[:,i-1]-k*v[:,i-1])*v[:,i-1]*dt
        
    return v,S,Q,X

In [ ]:
v1,S1,Q1,X1 = change_parameter(phi=0.000000001,alpha=0.001)
v2,S2,Q2,X2 = change_parameter(phi=0.001,alpha=0.001)
v3,S3,Q3,X3 = change_parameter(phi=0.01,alpha=0.001)
v4,S4,Q4,X4 = change_parameter(phi=0.1,alpha=0.001)


fig,ax = plt.subplots()


ax.plot(t,Q1[i],label="$\phi=0$")
ax.plot(t,Q2[i],label="$\phi=0.001$")
ax.plot(t,Q3[i],label="$\phi=0.01$")
ax.plot(t,Q4[i],label="$\phi=0.1$")

ax.set_title("Inventory over time with alpha = 0.001")
ax.set_xlabel("T")
ax.set_ylabel("Inventory")
ax.legend()
plt.show()

fig.savefig('change_phi_inventory.jpg')

In [ ]:
fig,ax = plt.subplots()


ax.plot(t,v1[i],label="$\phi=0$")
ax.plot(t,v2[i],label="$\phi=0.001$")
ax.plot(t,v3[i],label="$\phi=0.01$")
ax.plot(t,v4[i],label="$\phi=0.1$")

ax.set_title("Trading speed over time with alpha = 0.001")
ax.set_xlabel("T")
ax.set_ylabel("Trading speed")
ax.legend()
plt.show()

fig.savefig('change_phi_trading.jpg')

In [ ]:
v1,S1,Q1,X1 = change_parameter(phi=0.000000001,alpha=10**6)
v2,S2,Q2,X2 = change_parameter(phi=0.001,alpha=10**6)
v3,S3,Q3,X3 = change_parameter(phi=0.01,alpha=10**6)
v4,S4,Q4,X4 = change_parameter(phi=0.1,alpha=10**6)

fig,ax = plt.subplots()


ax.plot(t,Q1[i],label="$\phi=0$")
ax.plot(t,Q2[i],label="$\phi=0.001$")
ax.plot(t,Q3[i],label="$\phi=0.01$")
ax.plot(t,Q4[i],label="$\phi=0.1$")

ax.set_title("Inventory over time with alpha = 10^6")
ax.set_xlabel("T")
ax.set_ylabel("Inventory")
ax.legend()
plt.show()

fig.savefig('change_alpha_inventory.jpg')

In [ ]:
fig,ax = plt.subplots()


ax.plot(t,v1[i],label="$\phi=0$")
ax.plot(t,v2[i],label="$\phi=0.001$")
ax.plot(t,v3[i],label="$\phi=0.01$")
ax.plot(t,v4[i],label="$\phi=0.1$")

ax.set_title("Trading speed over time with alpha = 10^6")
ax.set_xlabel("T")
ax.set_ylabel("Trading speed")
ax.legend()
plt.show()

fig.savefig('change_alpha_trading.jpg')

In [ ]:
v1,S1,Q1,X1 = change_parameter(phi=0.000000001,alpha=0.001)
v2,S2,Q2,X2 = change_parameter(phi=0.001,alpha=0.001)
v3,S3,Q3,X3 = change_parameter(phi=0.01,alpha=0.001)
v4,S4,Q4,X4 = change_parameter(phi=0.1,alpha=0.001)

plt.figure(figsize=(15,7))

plt.subplot(121)

plt.plot(t,Q1[i],label="$\phi=0$")
plt.plot(t,Q2[i],label="$\phi=0.001$")
plt.plot(t,Q3[i],label="$\phi=0.01$")
plt.plot(t,Q4[i],label="$\phi=0.1$")
plt.title("Stock price over time")
plt.xlabel("T")
plt.ylabel("Inventory")
plt.legend()

plt.subplot(122)
plt.plot(t,v1[i],label="$phi=0$")
plt.plot(t,v2[i],label="$phi=0.001$")
plt.plot(t,v3[i],label="$phi=0.01$")
plt.plot(t,v4[i],label="$phi=0.1$")
plt.title("Stock price over time")
plt.xlabel("T")
plt.ylabel("Inventory")
plt.legend()

plt.show()

In [ ]:
v1,S1,Q1,X1 = change_parameter(phi=0.000000001,alpha=10**6)
v2,S2,Q2,X2 = change_parameter(phi=0.001,alpha=10**6)
v3,S3,Q3,X3 = change_parameter(phi=0.01,alpha=10**6)
v4,S4,Q4,X4 = change_parameter(phi=0.1,alpha=10**6)

plt.figure(figsize=(15,7))

plt.subplot(121)

plt.plot(t,Q1[i],label="$phi=0$")
plt.plot(t,Q2[i],label="$phi=0.001$")
plt.plot(t,Q3[i],label="$phi=0.01$")
plt.plot(t,Q4[i],label="$phi=0.1$")
plt.title("Stock price over time")
plt.xlabel("T")
plt.ylabel("Inventory")
plt.legend()

plt.subplot(122)
plt.plot(t,v1[i],label="$phi=0$")
plt.plot(t,v2[i],label="$phi=0.001$")
plt.plot(t,v3[i],label="$phi=0.01$")
plt.plot(t,v4[i],label="$phi=0.1$")
plt.title("Stock price over time")
plt.xlabel("T")
plt.ylabel("Inventory")
plt.legend()

plt.show()

## 2. Liqudation problem with order flow

**Model setup**

value function: $$H(t,x,S,\mu,q) = \sup_{\nu \in \mathscr{A}} \mathbb{E}_{t,x,S,\mu,q}[X_T^{\nu} + Q_T^{ \nu}(S_T^{\nu} - \alpha Q_T^{\nu}) - \phi \int_t^T(Q_u^{\nu})^2 du]$$

* $dX_t^{\nu} = (S_t^{\nu}-f(\nu_t)) \nu_t dt$
* $dS_t^{\nu} = (g(\mu_t^{+})-g(\mu_t^{-}+\nu_t) )dt + \sigma dW_t, \quad S_0^{\nu}=S$
* $d\mu_t^{+} = -\kappa \mu_t^{+}dt + \eta dL_t^{+}$
* $d\mu_t^{-} = -\kappa \mu_t^{-}dt + \eta dL_t^{-}$
* $dQ_t^{\nu} = - \nu_t dt, \quad Q_0^{\nu}=q$

HJB equation: $$\partial_t H + \frac{1}{2}\sigma^2 \partial_{SS}H +\mathscr{L}^{\mu}H - \phi q^2 + \sup_{\nu} \{\nu (S-f(\nu)) \partial_x H - (g(\mu_t^{+})-g(\mu_t^{-}+\nu_t)) \partial_S H - \nu \partial_q H \} = 0$$
$$H(T,x,S,q) = x + q(S-\alpha q)$$

* Consider the linear case: $f(\nu) = k\nu, g(\nu)=b\nu$

From the setting, the analytical solution is: $$ H(t,x,S,\mu,q) = x + qS + h_0(t,\mu) + qh_1(t,\mu) + q^2h_2(t,\mu) $$ 
where $$ h_2(t,\mu) = \chi(t) - \frac{1}{2}b, \quad \chi(t) = \sqrt{k\phi}\frac{1+\xi e^{2\gamma(T-t)}}{1-\xi e^{2\gamma(T-t)}}$$ $$h_1(t,\mu) = \mathscr{l}_0(t) + \mu \mathscr{l}_1(t)$$  $$\mathscr{l}_1(t) = b\bar {\mathscr{l}_1}(T-t)= \frac{1}{\xi e^{\gamma(T-t)}-e^{-\gamma(T-t)}} \{ e^{\gamma(T-t)} \xi \frac{1-e^{-(\kappa + \gamma)(T-t)}}{\kappa+\gamma} - e^{-\gamma(T-t)} \frac{1-e^{-(\kappa - \gamma)(T-t)}}{\kappa-\gamma} \}$$

Therefore, $$\nu_t^{*} = -\frac{1}{k} \chi(t) Q_t^{\nu^{*}} - \frac{b}{2k} \bar{\mathscr{l}_1}(t) \mu_t $$ where $\gamma = \sqrt{\frac{\phi}{k}}, \xi = \frac{\alpha - \frac{1}{2}b + \sqrt{k\phi}}{\alpha - \frac{1}{2}b - \sqrt{k\phi}}$.

In [ ]:
# First, simulate the poisson process
lambda1 = 2
num_events_simulated = 10

def Poisson_Process(mu, num_events):
    # use Inverse method 
    time_intervals = -np.log(np.random.random(num_events)) / mu
    total_events = time_intervals.cumsum()
    events = pd.DataFrame(np.ones(num_events), index=total_events)
    events[0] = events[0].cumsum()

    return events

events = Poisson_Process(lambda1, num_events_simulated)

plt.plot(events, marker='o',drawstyle='steps-post')
plt.title("Simulated Poisson Process")
plt.xlabel("time")
plt.ylabel("events")
plt.show()

In [ ]:
N = 7*36000
T = 1; dt = float(T) / N; t = np.linspace(0,T,N+1)

In [ ]:
lambda1 = 1000
num_events_simulated = 1500
L_positive = Poisson_Process(lambda1, num_events_simulated)
L_positive = L_positive[L_positive.index <= 1]
L_negative = Poisson_Process(lambda1, num_events_simulated)
L_negative = L_negative[L_negative.index <= 1]

dL_positive = np.zeros(len(t)-1)
dL_negative = np.zeros(len(t)-1)

for position in (L_positive.index / dt): 
    dL_positive[int(np.floor(position))] += 1

for position in (L_negative.index / dt): 
    dL_negative[int(np.floor(position))] += 1

In [ ]:
def chi(i):
    return np.sqrt(k*phi)*(1+epsilon*np.exp(2*gamma*(T-i*dt)))/(1-epsilon*np.exp(2*gamma*(T-i*dt)))

def l1(i):
    return (np.exp(gamma*(T-i*dt))*epsilon*(1-np.exp(-(kappa+gamma)*(T-i*dt)))/(kappa+gamma)-np.exp(-gamma*(T-i*dt))*(1-np.exp(-(kappa-gamma)*(T-i*dt)))/(kappa-gamma)) / (epsilon*np.exp(gamma*(T-i*dt))-np.exp(-gamma*(T-i*dt)))

In [ ]:
# Simulate the data 

# Run 1000 samples 
N_MC = 1000

# Set parameters
# trade once per second in one day (approximately set trade hour in one day is 7)
T = 1
N = 3600 * 7 
dt = float(T) / N
t = np.linspace(0,T,N+1)
# temporary price impact & permanent price impact
k = 10**(-3)
b = 10**(-4)
# terminal penalty
alpha = 100 * k
# running penalty
phi = 0.01
# volatility of stock price 
sigma = 0.1 
# initial price
S0 = 20
# initial inventory 
q = 1 

lambda1 = 1000
kappa = 10
# eta ~ Exp(5)
eta = np.random.exponential(5,(N_MC,len(t)))

dW = np.sqrt(dt)*np.random.randn(N_MC,N)
S = np.zeros([N_MC,len(t)])
v = np.zeros([N_MC,len(t)])
X = np.zeros([N_MC,len(t)])
Q = np.zeros([N_MC,len(t)])
mu = np.zeros([N_MC,len(t)])

Q[:,0] = q
S[:,0] = S0
X[:,0] = 0
mu[:,0] = 0

gamma = np.sqrt(phi/k)
epsilon = (alpha-0.5*b+np.sqrt(k*phi)) / (alpha-0.5*b-np.sqrt(k*phi))

In [ ]:
# Euler Scheme

# du_t = - kappa*u_t*dt + eta*(dL_t^{+} - dL_t^{-})
for i in range(N):
    mu[:,i+1] = mu[:,i] - kappa*mu[:,i]*dt + eta[:,i]*(dL_positive[i]-dL_negative[i]) 

for i in range(N):
    v[:,i] = - 1/k * chi(i) * Q[:,i] - b/(2*k) * l1(i) * mu[:,i]
    Q[:,i+1] = Q[:,i] - v[:,i]*dt
    S[:,i+1] = S[:,i] + b*(mu[:,i]-v[:,i])*dt + sigma*dW[:,i] 
    X[:,i+1] = X[:,i] + (S[:,i]-k*v[:,i])*v[:,i]*dt
    
v[:,N] = - 1/k * chi(i) * Q[:,N] - b/(2*k) * l1(i) * mu[:,N]

In [ ]:
# Plot
plt.figure(figsize=(20, 15))

plt.subplot(321)
for i in range(0,N_MC):
    plt.plot(t,S[i],linewidth=0.5)

plt.title("Stock price over time")
plt.xlabel("T")

plt.subplot(322)
for i in range(0,N_MC):
    plt.plot(t,X[i],linewidth=0.5)

plt.title("Revenue over time")
plt.xlabel("T")

plt.subplot(323)
for i in range(0,N_MC):
    plt.plot(t,v[i],linewidth=0.5)

plt.title("Optimal strategy over time")
plt.xlabel("T")
plt.ylabel("Trading Speed")


plt.subplot(324)
for i in range(0,N_MC):
    plt.plot(t,Q[i],linewidth=0.5)

plt.title("Inventory over time")
plt.xlabel("T")
plt.ylabel("Inventory")


plt.subplot(325)
for i in range(0,N_MC):
    plt.plot(t,mu[i],linewidth=0.5)

plt.title("Order flow over time")
plt.xlabel("T")
plt.ylabel("Order flow")

plt.show()

## 3. Liqudation problem with Limit orders

**Model setup**

value function: $$H(t,x,S,q) = \sup_{\delta \in \mathscr{A}} \mathbb{E}_{t,x,S,q}[X_{\tau}^{\delta} + Q_{\tau}^{\delta}(S_{\tau} - \alpha Q_{\tau}^{\delta})]$$

* $dX_t^{\delta} = (S_t + \delta_t) dN_t^{\delta}$
* $N_t^{\delta}$ denotes the controlled counting process corresponding to the number of market buy orders which lift the agent's offer. 
* $\delta_t$ denotes the depth at which the agent posts limit sell orders.
* $dS_t = \sigma dW_t, \quad S_0=S$
* $Q_t^{\nu} = \Pi - N_t^{\delta}, \quad Q_0=\Pi$

HJB equation: $$\partial_t H + \frac{1}{2}\sigma^2 \partial_{SS}H + \sup_{\delta} \{ \lambda e^{-k\delta}[H(t,x+(S+\delta),S,q-1) - H(t,x,S,q)]  \} = 0$$
$$H(T,x,S,q) = x + q(S-\alpha q)$$
$$H(t,x,S,0) = x$$

From the setting, the analytical solution is: $$ \delta^{*}(t,q) = \frac{1}{\kappa} + [h(t,q)-h(t,q-1)]$$ 
where $$ \omega(t,q) = \sum_{n=0}^{q} \frac{\tilde{\lambda}^n }{n!}e^{-\kappa \alpha (q-n)^2}(T-t)^n $$
$$ h(t,q) = \frac{1}{\kappa} log \omega(t,q) $$
$$ \tilde{\lambda} = \lambda e^{-1} $$

In [ ]:
# Simulate the data 
# Run 5 samples 
N_MC = 3

# Set parameters
# set T = 60 seconds
T = 1
N = 60
dt = float(T) / N
t = np.linspace(0,T,N+1)
# terminal penalty
alpha = 0.001
# volatility of stock price 
sigma = 0.01 
# initial price
S0 = 30
# initial inventory 
Pi = 5

lambda1 = 50
kappa = 100

dW = np.sqrt(dt)*np.random.randn(N_MC,N)
S = np.zeros([N_MC,len(t)])
X = np.zeros([N_MC,len(t)])
Q = np.zeros([N_MC,len(t)])

# Here I consider a 3-d array to contain 1. the number of samples, 2. time , 3. inventory q
omega = np.zeros((N_MC,len(t),Pi+1))
omega[:,:,0] = 1
delta = np.zeros((N_MC,len(t),Pi))

Q[:,0] = Pi
S[:,0] = S0
X[:,0] = 0

lambda_tilde = lambda1*np.exp(-1)

In [ ]:
# Calculate the optimal depth

for i in range(N+1):
    for q in range(1,Pi+1):
        for n in range(q+1):
            omega[:,i,q] += lambda_tilde**n * np.exp(-kappa*alpha*(q-n)**2) * (T-i*dt)**n / np.math.factorial(n)
    for q in range(1,Pi+1):
        delta[:,i,q-1] = (1+np.log(omega[:,i,q]/omega[:,i,q-1])) / kappa

In [ ]:
fig,ax = plt.subplots()

ax.plot(delta[2])
ax.set_xlabel("T(sec.)")
ax.set_ylabel("Optimal depth")
plt.show()

fig.savefig('depth_model3.jpg')

In [ ]:
# Euler Scheme
for i in range(N):
    S[:,i+1] = S[:,i] +  sigma*dW[:,i] 

In [ ]:
M = []
num_events_simulated = 200

for i in range(N_MC):
    L_positive = Poisson_Process(lambda1, num_events_simulated)
    L_positive = L_positive[L_positive.index <= 1]
    dL_positive = np.zeros(len(t))
    for position in (L_positive.index / dt): 
        dL_positive[int(np.floor(position))] += 1
    M.append(dL_positive)

In [ ]:
# Simulate dN_t

P = np.exp(-kappa*delta)
Prob = np.random.binomial(1,P)

for i in range(N_MC):
    for j in range(Pi):
        Prob[i,:,j] = M[i]*Prob[i,:,j]  
        
        
for iteration in range(N_MC):
    for i in range(N):
        q = int(Q[iteration,i])
        Q[iteration,i+1] = max(Q[iteration,i] - Prob[iteration,i,q-1],0)

In [ ]:
optimal_depth = []

for iteration in range(N_MC):
    depth = []
    for i in range(N+1):
        q = int(Q[iteration,i])
        if q != 0:
            depth.append(delta[iteration,i,q-1])
        else: 
            depth.append(np.nan)
    optimal_depth.append(depth)

In [ ]:
fig,ax = plt.subplots()

for i in range(N_MC):
    ax.plot(optimal_depth[i])

ax.set_title("Optimal depth over time")
ax.set_xlabel("T(sec.)")
ax.set_ylabel("Depth")
plt.show()

fig.savefig('optimal_depth_model3.jpg')

In [ ]:
# Plot the inventory process

fig,ax = plt.subplots()

for i in range(N_MC):
    ax.plot(Q[i,:])

ax.set_title("Inventory over time")
ax.set_xlabel("T(sec.)")
ax.set_ylabel("Inventory")
plt.show()

fig.savefig('inventory_model3.jpg')

In [ ]:
fig,ax = plt.subplots()

for i in range(N_MC):
    ax.plot(t,S[i,:])

ax.set_title("Stock price over time")
ax.set_xlabel("T(sec.)")
ax.set_ylabel("Stock price")
plt.show()


fig.savefig('stockprice_model3.jpg')

In [ ]:
for iteration in range(N_MC):
    for i in range(N):
        X[iteration,i+1] = X[iteration,i] + (S[iteration,i]+optimal_depth[iteration][i])*(Q[iteration,i]-Q[iteration,i+1])

In [ ]:
for i in range(N_MC):
    for i in range(N):
        if np.isnan(X[iteration,i+1]):
            X[iteration,i+1] = X[iteration,i]

In [ ]:
fig,ax = plt.subplots()

for i in range(N_MC):
    ax.plot(t,X[i,:])

ax.set_title("Revenues over time")
ax.set_xlabel("T(sec.)")
ax.set_ylabel("Revenue")
plt.show()

fig.savefig('cash_model3.jpg')